In [1]:
import os
import re
import json
import math
import random
import shutil
import hashlib

from pathlib import Path
from collections import defaultdict, Counter

import numpy as np
import pandas as pd

import cv2
from PIL import Image

import matplotlib.pyplot as plt

from tqdm import tqdm

1. Dataset Freeze

In [2]:
SRC_DATASET = Path("dataset")
WORK_DATASET = Path("dataset_working")


if WORK_DATASET.exists():
    raise RuntimeError(
        "dataset_raw or dataset_working already exists.\n"
        "Delete them manually if you want to re-freeze."
    )

print("📦 Creating dataset_working (preprocessing copy)...")
shutil.copytree(SRC_DATASET, WORK_DATASET)

print("• dataset_working  → use for preprocessing")

📦 Creating dataset_working (preprocessing copy)...
• dataset_working  → use for preprocessing


In [3]:
import shutil
import numpy as np
from pathlib import Path
from collections import defaultdict
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

# ========= CONFIG =========
DATASET_ROOT = Path("dataset_working")
OUTPUT_ROOT = Path("dataset_split")

TRAIN_RATIO = 0.70
VAL_RATIO = 0.15
TEST_RATIO = 0.15

SEED = 42
COPY_FILES = True
IMG_EXTS = {".jpg", ".jpeg", ".png"}
# ==========================

assert abs(TRAIN_RATIO + VAL_RATIO + TEST_RATIO - 1.0) < 1e-6

IMG_DIR = DATASET_ROOT / "images"
LBL_DIR = DATASET_ROOT / "labels"

# Create output folders
for split in ["train", "val", "test"]:
    (OUTPUT_ROOT / split / "images").mkdir(parents=True, exist_ok=True)
    (OUTPUT_ROOT / split / "labels").mkdir(parents=True, exist_ok=True)

# ------------------------------------------------
# 1. Parse dataset → multilabel vectors
# ------------------------------------------------
samples = []
class_ids = set()

for img_path in IMG_DIR.iterdir():
    if img_path.suffix.lower() not in IMG_EXTS:
        continue

    label_path = LBL_DIR / f"{img_path.stem}.txt"
    if not label_path.exists():
        print(f"⚠️ Missing label: {img_path.name}")
        continue

    labels = set()
    with open(label_path) as f:
        for line in f:
            cls = int(line.split()[0])
            labels.add(cls)
            class_ids.add(cls)

    samples.append({
        "img": img_path,
        "lbl": label_path,
        "classes": labels
    })

class_ids = sorted(class_ids)
class_to_idx = {c: i for i, c in enumerate(class_ids)}

X = np.arange(len(samples))
Y = np.zeros((len(samples), len(class_ids)), dtype=int)

for i, s in enumerate(samples):
    for c in s["classes"]:
        Y[i, class_to_idx[c]] = 1

# ------------------------------------------------
# 2. Train vs Temp split (70 / 30)
# ------------------------------------------------
msss_1 = MultilabelStratifiedShuffleSplit(
    n_splits=1,
    test_size=(VAL_RATIO + TEST_RATIO),
    random_state=SEED
)

train_idx, temp_idx = next(msss_1.split(X, Y))

# ------------------------------------------------
# 3. Val vs Test split (15 / 15)
# ------------------------------------------------
Y_temp = Y[temp_idx]

msss_2 = MultilabelStratifiedShuffleSplit(
    n_splits=1,
    test_size=TEST_RATIO / (VAL_RATIO + TEST_RATIO),
    random_state=SEED
)

val_idx_rel, test_idx_rel = next(msss_2.split(
    np.arange(len(temp_idx)),
    Y_temp
))

val_idx = temp_idx[val_idx_rel]
test_idx = temp_idx[test_idx_rel]

# ------------------------------------------------
# 4. File transfer
# ------------------------------------------------
def transfer(indices, split):
    for i in indices:
        img = samples[i]["img"]
        lbl = samples[i]["lbl"]

        dst_img = OUTPUT_ROOT / split / "images" / img.name
        dst_lbl = OUTPUT_ROOT / split / "labels" / lbl.name

        if COPY_FILES:
            shutil.copy2(img, dst_img)
            shutil.copy2(lbl, dst_lbl)
        else:
            shutil.move(img, dst_img)
            shutil.move(lbl, dst_lbl)

transfer(train_idx, "train")
transfer(val_idx, "val")
transfer(test_idx, "test")

# ------------------------------------------------
# 5. Summary
# ------------------------------------------------
print("✅ Multi-label stratified split completed")
print(f"Train: {len(train_idx)}")
print(f"Val  : {len(val_idx)}")
print(f"Test : {len(test_idx)}")
print(f"Total: {len(samples)}")


✅ Multi-label stratified split completed
Train: 439
Val  : 92
Test : 101
Total: 632


In [ ]:
from pathlib import Path
from collections import defaultdict

DATASET_SPLIT = Path("dataset_split")
NUM_CLASSES = 4   

def analyze_split(split_name):
    label_dir = DATASET_SPLIT / split_name / "labels"
    image_dir = DATASET_SPLIT / split_name / "images"

    box_counts = defaultdict(int)
    image_presence = defaultdict(int)

    for label_file in label_dir.glob("*.txt"):
        with open(label_file, "r") as f:
            lines = f.readlines()

        present_classes = set()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue

            class_id = int(parts[0])
            box_counts[class_id] += 1
            present_classes.add(class_id)

        for cid in present_classes:
            image_presence[cid] += 1

    total_images = len(list(image_dir.glob("*")))

    print(f"\n📦 SPLIT: {split_name.upper()}")
    print(f"Total images: {total_images}")

    print("\n📊 Bounding Box Count:")
    for cid in range(NUM_CLASSES):
        print(f"  Class {cid}: {box_counts[cid]} boxes")

    print("\n🖼️ Image-level Presence:")
    for cid in range(NUM_CLASSES):
        count = image_presence[cid]
        pct = (count / total_images * 100) if total_images else 0
        print(f"  Class {cid}: {count} images ({pct:.1f}%)")

# Run for all splits
for split in ["train", "val", "test"]:
    analyze_split(split)



📦 SPLIT: TRAIN
Total images: 439

📊 Bounding Box Count:
  Class 0: 269 boxes
  Class 1: 262 boxes
  Class 2: 233 boxes
  Class 3: 200 boxes

🖼️ Image-level Presence:
  Class 0: 200 images (45.6%)
  Class 1: 162 images (36.9%)
  Class 2: 195 images (44.4%)
  Class 3: 110 images (25.1%)

📦 SPLIT: VAL
Total images: 92

📊 Bounding Box Count:
  Class 0: 58 boxes
  Class 1: 58 boxes
  Class 2: 51 boxes
  Class 3: 39 boxes

🖼️ Image-level Presence:
  Class 0: 43 images (46.7%)
  Class 1: 34 images (37.0%)
  Class 2: 42 images (45.7%)
  Class 3: 23 images (25.0%)

📦 SPLIT: TEST
Total images: 101

📊 Bounding Box Count:
  Class 0: 59 boxes
  Class 1: 63 boxes
  Class 2: 60 boxes
  Class 3: 32 boxes

🖼️ Image-level Presence:
  Class 0: 43 images (42.6%)
  Class 1: 35 images (34.7%)
  Class 2: 42 images (41.6%)
  Class 3: 24 images (23.8%)
